In [1]:
import tensorflow as tf
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [3]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

#### Build Computation Graph
* he initialization
* custom activation: leaky relu
* gradient clipping before optimization step
* regularization as parameter to layer object

In [4]:
def leaky_relu(z, name=None):
    return tf.maximum(0.01 * z, z, name=name)

In [5]:
# can also use dense layer
with tf.name_scope("dnn"):
    # dense layer uses xavier initialization by default, can change to he initialization
    he_init = tf.contrib.layers.variance_scaling_initializer()
    
    # exponential 
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.elu, 
                              kernel_initializer=he_init, kernel_regularizer=tf.contrib.layers.l1_regularizer(scale=0.001))
    
    # plug in leaky_relu as activation
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=leaky_relu,
                             kernel_regularizer=tf.contrib.layers.l1_regularizer(scale=0.001))
    
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs", activation=tf.nn.relu)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [7]:
# learning_rate = 0.01
threshold = 1

with tf.name_scope("train"):
    # learning rate scheduling
    starter_learning_rate = 0.1
    decay_step = 100000
    decay_rate = 1/10
    global_step = tf.Variable(0, trainable=False, name="global_step")
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           decay_step, decay_rate, staircase=True)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
    # other optimizer
    # optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    # optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9, use_nesterov=True)
    # optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
    # optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate, momentum=0.9, decay=0.9, epsilon=1e-10)
    # optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    # clip gradient before update!
    # grads_and_vars = optimizer.compute_gradients(loss)
    # capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var) for grad, var in grads_and_vars]
    # training_op = optimizer.apply_gradients(capped_gvs)
    
    training_op = optimizer.minimize(loss, global_step=global_step)

In [8]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

#### Execution Phase

In [10]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [11]:
n_epochs = 10
batch_size = 50

In [12]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for interaction in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            feed = {X: X_batch, y: y_batch}
            sess.run(training_op, feed_dict=feed)
        acc_train = accuracy.eval(feed_dict=feed)
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        print("epoch %i, train acc %.5f, val acc %.5f"%(epoch, acc_train, acc_val))
    saver.save(sess, "outputs/final_model.ckpt")

epoch 0, train acc 1.00000, val acc 0.95440
epoch 1, train acc 0.98000, val acc 0.96180
epoch 2, train acc 1.00000, val acc 0.97260
epoch 3, train acc 0.98000, val acc 0.97520
epoch 4, train acc 0.98000, val acc 0.97760
epoch 5, train acc 1.00000, val acc 0.97520
epoch 6, train acc 1.00000, val acc 0.97860
epoch 7, train acc 1.00000, val acc 0.97580
epoch 8, train acc 1.00000, val acc 0.98020
epoch 9, train acc 1.00000, val acc 0.97860


#### Test

In [13]:
with tf.Session() as sess:
    saver.restore(sess, "outputs/final_model.ckpt")
    Z = logits.eval(feed_dict={X: mnist.test.images})
    acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
acc_test

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from outputs/final_model.ckpt


0.9776